In [10]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore')

## データの読み込み

In [186]:
# データの読み込み
# df_train_org = pd.read_csv('./data/preprocessed_train.csv')
# df_test_org = pd.read_csv('./data/preprocessed_test.csv')
df_train_org = pd.read_csv('./data/train.csv')
df_test_org = pd.read_csv('./data/test.csv')


In [187]:
# データ量が多いのでコーディング中はサンプリング
# df = df_org.sample(100000, replace=True)
df = df_train_org
df_test = df_test_org

In [188]:
print(df.shape)
print(df_test.shape)

(1200, 34)
(800, 33)


In [189]:
# ここでNaNが多いものはできれば除きたい
# df.isnull().sum()

## データの準備

In [190]:
df_test.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,34,Travel_Frequently,358.039056,Research & Development,9.926847,3,Life Sciences,843,1,...,3,80,1,11,2,3,7,7,2,7
1,2,37,Travel_Rarely,749.376719,Research & Development,10.391959,3,Medical,1361,1,...,4,80,0,5,3,2,1,3,1,0
2,4,34,Travel_Frequently,958.484177,Research & Development,3.297025,3,Life Sciences,975,2,...,3,80,1,21,2,2,4,3,1,4
3,5,34,Travel_Rarely,79.174592,Research & Development,2.430032,4,Medical,1591,3,...,4,80,3,9,3,3,20,6,1,4
4,6,29,Travel_Rarely,943.191218,Research & Development,10.017149,2,Life Sciences,1803,3,...,3,80,1,25,2,4,1,7,0,0


In [191]:
y_train = df[['Attrition']]
# x_cols = ['class', 'country', 'period', 'n_words', 'log_goal', 'launched_year', 'launched_month', 'launched_day', 
#          'name_vowel_ratio', 'Goal_1000', 'Goal_500', 'Goal_10', 'median_goal_Last_Week']
# print(df.columns)
# columns = df.columns.drop([])
# x_cols = df.columns.drop(['Unnamed: 0','id','Attrition', "Over18", "StandardHours"])
x_cols = df.columns.drop(['Attrition', "Over18", "StandardHours", "JobInvolvement", "PerformanceRating"])
x_train = df[x_cols]
x_test = df_test[x_cols]

In [192]:
x_train.head(2)

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,13,3,0,1,3,2,11,8,7,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,11,3,1,20,1,1,5,9,4,4


In [193]:
# x_cols_binning = ['BusinessTravel', 'Department']
# binning = KBinsDiscretizer(n_bins=5)

In [194]:
# x_binned = binning.fit_transform(x_cols_binning)

## モデル学習（単体）

In [195]:
# # https://www.kaggle.com/code/yairhadad1/kickstarter-predict-project-success-roc-0-78/notebook
# # 上記のノートブックを参考にしてる解法
# from sklearn.base import BaseEstimator, TransformerMixin
# class ItemSelector(BaseEstimator, TransformerMixin):
#     def __init__(self, key):
#         self.key = key

#     def fit(self, x, y=None):
#         return self

#     def transform(self, data_dict):
#         return data_dict[self.key]

In [196]:
# # https://kakakakakku.hatenablog.com/entry/2021/11/29/084330
# # Pipelineを使うことで、前処理やML学習などを１つのオブジェクトとして扱え、可読性高く実装できる。
# from sklearn.pipeline import Pipeline

# # # split
# # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train)

# # # name
# # from sklearn.feature_extraction.text import CountVectorizer  
# # name_pipeline = Pipeline([('name_column', ItemSelector(key = 'name')),
# #                           ('vectorizer', CountVectorizer(max_features=100))])

# # dummeys
# from sklearn.preprocessing import OneHotEncoder
# one_hot_pipeline = Pipeline([('hot_columns', ItemSelector(key = ['BusinessTravel','Department',
#                                                                  'EducationField', 'Gender', 'JobRole',
#                                                                 'MaritalStatus'])),
#                              ('oneHowEncoder', OneHotEncoder(handle_unknown='ignore',sparse=True))])

# # # min max scaler
# # from sklearn.preprocessing import MinMaxScaler
# # min_max_pipeline = Pipeline([('min_max_columns', ItemSelector(key = ['project_time','word_count',
# #                                                                      'goal_per_day', 'n_words', 'name_len'])),
# #                              ('minMaxScaler', MinMaxScaler())])

# # Binning（連続的な特徴量を離散化する）
# from sklearn.preprocessing import KBinsDiscretizer
# k_bins_pipeline = Pipeline([('goal', ItemSelector(key = ['Age', 'DailyRate', 'HourlyRate', 'MonthlyIncome',
#                                                          'DistanceFromHome', 'EmployeeNumber', 'TotalWorkingYears',
#                                                          'YearsAtCompany', 'YearsInCurrentRole',
#                                                          'YearsSinceLastPromotion', 'YearsWithCurrManager'])),
#                              ('k_bins', KBinsDiscretizer(n_bins = 4,encode = 'onehot',strategy = 'quantile'))])

# # FeatureUnion
# from sklearn.pipeline import FeatureUnion
# feature_pipeline = FeatureUnion([('one_hot',one_hot_pipeline),
#                                  # ('name',name_pipeline),
#                                  # ('min_max',min_max_pipeline),
#                                  ('k_bins',k_bins_pipeline)])

# feature_pipeline.fit(x_train)

# x_train = feature_pipeline.transform(x_train)
# x_test = feature_pipeline.transform(x_test)


In [197]:
x_train

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,13,3,0,1,3,2,11,8,7,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,11,3,1,20,1,1,5,9,4,4
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,18,2,1,4,3,3,3,2,0,2
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,20,4,1,6,0,1,5,4,7,7
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,...,14,2,1,8,3,3,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1988,49,Travel_Rarely,969.251891,Research & Development,13.243579,2,Medical,1587,4,...,18,2,0,33,3,2,3,2,1,2
1196,1994,37,Travel_Frequently,437.940367,Research & Development,2.339438,1,Other,137,4,...,11,4,0,6,5,2,9,2,0,8
1197,1996,40,Non-Travel,978.883360,Human Resources,10.214979,3,Life Sciences,1587,3,...,12,2,1,19,2,3,14,9,9,9
1198,1997,37,Travel_Frequently,170.494984,Sales,2.639879,2,Marketing,120,1,...,16,1,2,6,5,3,3,2,0,3


In [198]:
# 数値エンコーディング（GBDT以外の場合は必須）
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train = x_train.apply(encoder.fit_transform)
# x_test = x_test.drop(['id']).apply(encoder.fit_transform)
x_test = x_test.apply(encoder.fit_transform)

In [199]:
from sklearn.model_selection import train_test_split

# # 訓練データとテストデータを8:2に分割
# # ※評価を揃えるため df 以外の引数は変更しないこと
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1234)

In [200]:
import lightgbm as lgb
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

# パラメータ
RANDOM_STATE = 10
num_round = 1000
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=TEST_SIZE,random_state=RANDOM_STATE)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

In [201]:
x_train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1011,1011,0,2,15,1,1067,1,1,134,1,...,2,2,0,6,2,0,9,2,7,7
924,924,5,2,128,2,85,2,2,7,0,...,2,2,2,1,5,2,1,0,0,0
654,654,15,1,41,1,106,3,5,126,3,...,0,3,0,33,3,2,2,2,2,2
716,716,28,2,672,1,999,3,1,51,2,...,0,1,1,6,0,1,5,2,0,2
681,681,7,2,340,1,551,1,1,145,2,...,6,3,1,1,3,1,7,7,0,0


In [202]:
# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 50,
    'verbose': 0
}

In [203]:
# 訓練データに対する予測
# https://nigimitama.hatenablog.jp/entry/2021/01/05/205741
# No further splits with positive gain, best gain: -infが表示された場合、
# 「pre-pruningが行われたんだな」と考えて、min_samples_in_leaf などの別のハイパーパラメータを調整する手がかりにすればよいかもしれない
# LightGBM学習
model = lgb.train(params, lgb_train, num_round, 
                  valid_names=['train', 'valid'], valid_sets=lgb_eval,
                  early_stopping_rounds=20, verbose_eval=False)

# LightGBM推論
y_pred = model.predict(x_valid, num_iteration=model.best_iteration)

# 評価
y_pred = y_pred.round(0)
# acc_train = accuracy_score(y_test, y_pred)
acc_train = accuracy_score(y_true=y_valid, y_pred=y_pred)

print(f"訓練データの正答率: {100 * acc_train:.3f} %")    

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000635 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

In [204]:
# テストデータに対する予測
# LightGBM推論
y_test_pred = model.predict(x_test, num_iteration=model.best_iteration)
y_test_pred = y_test_pred.round(0).astype(int)

# # 評価
# acc_train = accuracy_score(y_true=y_test, y_pred=y_pred)
# print(f"テストデータの正答率: {100 * acc_train:.3f} %")  

# # print(y_test)
# # print(y_pred)

In [205]:
# a=1.0
# int(a)

In [206]:
pd_y = pd.DataFrame(y_test_pred)
pd_id = pd.DataFrame(df_test['id'])
df_submit = pd.DataFrame({'col_list':df_test.id, 'col_array':y_test_pred})

In [207]:
df_submit.to_csv('./data/submit_2.csv', header=False, index = False)

In [208]:
# feature importance for Light GBM
predictor_columns = x_train.columns
importance = pd.DataFrame(model.feature_importance(), index=predictor_columns, columns=['importance'])
# display(importance)
display(importance.sort_values('importance', ascending = False))

# predictor_columns = x_train.columns
# feat_import = list(zip(predictor_columns, list(model.feature_importances())))
# ns_df = pd.DataFrame(data = feat_import, columns=['Feat_names', 'Importance'])
# ns_df_sorted = ns_df.sort_values(['Importance', 'Feat_names'], ascending = [False, True])

# ns_df_sorted

,importance
DistanceFromHome,62
EmployeeNumber,54
id,52
HourlyRate,51
DailyRate,46
MonthlyIncome,42
TotalWorkingYears,40
Age,39
YearsWithCurrManager,36
JobRole,32


## 交差検証

In [209]:
# !pip install lightgbm

In [210]:
from sklearn.model_selection import KFold
n_split = 5 # グループ数を設定（今回は5分割）
seed = 1234
# cross_valid_mae = 0
split_num = 1
kf = KFold(n_splits=n_split, shuffle=True, random_state=seed)

In [211]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import lightgbm as lgb

# scores_train = []
scores_valid = []
models = []
itr = 1

for train_idx, valid_idx in kf.split(x_train, y_train):
    x_train, y_train = x_train.iloc[train_idx], y_train.iloc[train_idx]
    print(train_idx)
    print(valid_idx)
    x_valid, y_valid = x_train.iloc[valid_idx], y_train.iloc[valid_idx]
    
    # lightgbmのデータ構造に変換
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_valid, y_valid)
    
    # ハイパーパラメータの設定
    # metricsはaccuracy_scoreにすべきかも
    params = {'num_leaves': 64, 'objective': 'binary', 'seed': seed, 
              'verbose': 0, 'metrics': 'binary_logloss'}
    num_round = 1000

    # 学習の実行
    # カテゴリ変数を指定した場合の挙動はKaggle本 p240参照
    # categorical_features = ['category', 'main_category', 'currency', 'country']
    model = lgb.train(params, lgb_train, num_round, 
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_valid],
                     early_stopping_rounds=20, verbose_eval=False)
    
    # # 訓練データに対する予測、正答率
    # y_pred = clf.predict(x_train)
    # acc_train = accuracy_score(y_true=y_train, y_pred=y_pred)
    # print(f"{itr:02}train_訓練データの正答率: {100 * acc_train:.2f} %")
    # scores_train.append(acc_train)  
    
    # 検証データに対する予測
    # accurasy_scoreするには0で丸め込みが必要（https://shiokoji11235.com/two_interface_of_lightgbm）
    y_pred = model.predict(x_valid)
    y_pred = y_pred.round(0)

    score_valid = accuracy_score(y_valid, y_pred)
    print(f"{itr:02}valid_accuracy_score: {score_valid:.4f} %")    

    # score_valid = log_loss(y_true=y_valid, y_pred=y_pred)
    # print(f"{itr:02}valid_logloss: {score_valid:.4f} %")    
    # score_valid = roc_auc_score(y_valid, y_pred)
    # print(f"{itr:02}valid_auc_score: {score_valid:.4f} %")    

    scores_valid.append(score_valid)    
    models.append(model)
    
    itr += 1

[  0   1   2   3   4   5   6   8  10  12  13  14  15  16  17  18  19  20
  22  23  24  25  26  27  28  30  31  32  33  34  35  36  37  38  39  41
  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  63
  64  66  68  69  70  71  73  74  75  76  77  79  80  81  82  84  85  86
  87  88  90  91  92  95  96  97  98  99 100 101 102 103 105 106 107 110
 111 113 114 115 116 117 119 120 121 123 125 126 127 129 130 131 132 133
 134 135 136 138 139 140 142 144 145 147 149 150 151 152 153 154 157 158
 161 162 163 164 165 167 168 169 170 171 172 173 174 176 177 178 179 180
 181 182 183 184 185 186 187 189 190 191 193 194 195 196 197 198 199 200
 201 202 203 204 207 208 209 210 211 212 213 215 216 217 218 220 221 223
 224 225 228 229 231 232 233 235 236 237 238 239 240 241 242 243 244 246
 247 249 250 251 252 253 254 256 257 258 259 260 261 263 264 265 268 269
 271 272 273 274 275 278 279 280 283 284 286 288 289 291 292 293 294 295
 296 297 299 300 301 302 303 304 305 306 308 309 31

IndexError: positional indexers are out-of-bounds

In [212]:
# 各foldのスコアの平均
# print(np.mean(scores_train))
print(np.mean(scores_valid))

nan


## 推論

In [153]:
y_preds = np.zeros(len(x_test))

for model in models:
    y_pred = model.predict(x_test)
    y_pred = y_pred.round(0)
    y_preds += y_pred / len(models)
    
# print(y_test)
# print('***')
# print(y_preds)
# print('***')
# print(y_pred)

y_test_preds = y_preds.round(0).astype(int)
# score_test = accuracy_score(y_test, y_preds)
# print(f"test_accuracy_score: {score_test:.4f} %")    

[LightGBM] [Fatal] The number of features in data (28) is not the same as it was in training data (30).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.


LightGBMError: The number of features in data (28) is not the same as it was in training data (30).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [123]:
# # テストデータに対する予測
# # LightGBM推論
# y_test_pred = model.predict(x_test, num_iteration=model.best_iteration)
# y_test_pred = y_test_pred.round(0).astype(int)


In [124]:
pd_y = pd.DataFrame(y_test_preds)
pd_id = pd.DataFrame(df_test['id'])
df_submit = pd.DataFrame({'col_list':df_test.id, 'col_array':y_test_preds})

In [125]:
df_submit.to_csv('./data/submit_3.csv', header=False, index = False)